In [1]:
import csv
import sys
import re
import os
import numpy as np

from astropy.io import fits
import astropy.units as u

if sys.platform=='darwin':
    top_path='/Users/codydirks/PGCC/'
else:
    top_path='/DataDisk/datafiles/PGCC_HST/'

x1d_dir=top_path+'x1d_files/'

def load_results(filename):
    gaia_filenames=[top_path+'gaia_data/TgasSource_000-000-0'+'{:02}'.format(i)+'.fits' for i in range(16)]
    pgcc_data=fits.open(top_path+'HFI_PCCS_GCC_R2.02.fits')[1].data
    sightline_pgcc_gaia_data=[]
    with open(filename,'r') as myfile:
        for line in myfile:
            dat=line.strip('|\n').split('|')
            sightline=dat[0]
            coords=dat[1].split(' ')
            ra=float(coords[0])*u.degree
            dec=float(coords[1])*u.degree
            pgcc=pgcc_data[int(dat[2])]
            if dat[3] != 'None':
                fl,idx=map(int,dat[3][1:-1].split(','))
                gaia_entry=fits.open(gaia_filenames[fl])[1].data[idx]
            else:
                gaia_entry=None
                
            sightline_pgcc_gaia_data.append([sightline,(ra,dec),pgcc,gaia_entry])
    return sightline_pgcc_gaia_data

In [2]:
with open('primary_sightlines.csv','rb') as csvfile:
    inp=csv.reader(csvfile)
    primary_sightlines=inp.next()
    
with open('secondary_sightlines.csv','rb') as csvfile:
    inp=csv.reader(csvfile)
    secondary_sightlines=inp.next()
    
    
filename='sightline_pgcc_gaia_results.txt'
all_data=load_results(filename)
all_data.sort(key=lambda x: (x[0][0],int(re.search(r'\d+',x[0]).group())))
primary_sample=[x for x in all_data if x[0] in primary_sightlines]
secondary_sample=[x for x in all_data if x[0] in secondary_sightlines]

In [3]:
lines=[(1235.838,'KrI'), # KrI
      (1237.0589,'GeII'), # GeII
      (1239.9253,'MgII'), # MgII
      (1317.2170,'NiII'), # NiII
      (1355.5977,'OI'),   # OI
      ]

lines=[(1280.1353,'C_I'),
      (1328.8333,'C_I')]
primary_sightlines.sort(key=lambda x: int(float(x[2:])))
for sl in primary_sightlines:
    print sl
    x1ds=[x for x in os.listdir(x1d_dir+sl+'/E140H/') if x.endswith('_x1d.fits')]
    pars=[x for x in os.listdir(x1d_dir+sl+'/E140H/') if x.endswith('.par')]
    txts=[x for x in os.listdir(x1d_dir+sl+'/E140H/') if x.endswith('.txt')]
    for line in lines:
        for x1d in x1ds:
            hdu=fits.open(x1d_dir+sl+'/E140H/'+x1d)
            minwave,maxwave=hdu[1].data[-1][2][0],hdu[1].data[0][2][-1]
            if minwave < line[0] < maxwave:
                print any([str(int(line[0])) in x for x in pars])
                print '\t', line[1],line[0]#, x1d.strip('_x1d.fits')
                break
        
    print

HD13841
True
	C_I 1280.1353
False
	C_I 1328.8333

HD23180
True
	C_I 1328.8333

HD23478
True
	C_I 1280.1353
False
	C_I 1328.8333

HD24398
True
	C_I 1328.8333

HD24534
True
	C_I 1280.1353
True
	C_I 1328.8333

HD25443
True
	C_I 1280.1353
False
	C_I 1328.8333

HD43582
True
	C_I 1280.1353
False
	C_I 1328.8333

HD62542
True
	C_I 1280.1353
False
	C_I 1328.8333

HD72350
True
	C_I 1280.1353
False
	C_I 1328.8333

HD108927
True
	C_I 1280.1353
False
	C_I 1328.8333

HD112999
True
	C_I 1280.1353
True
	C_I 1328.8333

HD124314
True
	C_I 1280.1353
True
	C_I 1328.8333

HD147683
True
	C_I 1280.1353
True
	C_I 1328.8333

HD148594
True
	C_I 1280.1353
False
	C_I 1328.8333

HD148937
True
	C_I 1280.1353
True
	C_I 1328.8333

HD165918
False
	C_I 1280.1353
True
	C_I 1328.8333

HD185418
True
	C_I 1280.1353
False
	C_I 1328.8333

HD203532
True
	C_I 1280.1353
True
	C_I 1328.8333

HD208947
True
	C_I 1280.1353
True
	C_I 1328.8333

HD210839
False
	C_I 1280.1353
True
	C_I 1328.8333

HD232522
True
	C_I 1280.1353
False
	C_

In [ ]:
for sl in secondary_sightlines:
    print sl
    x1ds=[x for x in os.listdir(x1d_dir+sl+'/E140H/') if x.endswith('_x1d.fits')]
    for line in lines:
        for x1d in x1ds:
            hdu=fits.open(x1d_dir+sl+'/E140H/'+x1d)
            minwave,maxwave=hdu[1].data[-1][2][0],hdu[1].data[0][2][-1]
            if minwave < line[0] < maxwave:
                print '\t', line[1]#, x1d.strip('_x1d.fits')
                break
        print
    print